In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 128
tree_depth = 8
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.153863906860352 | KNN Loss: 6.230323791503906 | BCE Loss: 1.9235402345657349
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.229302406311035 | KNN Loss: 6.230156898498535 | BCE Loss: 1.999145746231079
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.185433387756348 | KNN Loss: 6.230011940002441 | BCE Loss: 1.9554216861724854
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.193683624267578 | KNN Loss: 6.230076313018799 | BCE Loss: 1.9636069536209106
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.184626579284668 | KNN Loss: 6.2299370765686035 | BCE Loss: 1.9546892642974854
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.163071632385254 | KNN Loss: 6.229653358459473 | BCE Loss: 1.9334180355072021
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.137682914733887 | KNN Loss: 6.229648113250732 | BCE Loss: 1.9080350399017334
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.162541389465332 | KNN Loss: 6.229461669921875 | BCE Loss: 1.93307

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.0865478515625 | KNN Loss: 5.937524795532227 | BCE Loss: 1.1490232944488525
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.04889440536499 | KNN Loss: 5.923275470733643 | BCE Loss: 1.1256190538406372
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.000793933868408 | KNN Loss: 5.882357597351074 | BCE Loss: 1.118436336517334
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.964555740356445 | KNN Loss: 5.85343074798584 | BCE Loss: 1.1111252307891846
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 6.921308517456055 | KNN Loss: 5.802789211273193 | BCE Loss: 1.1185193061828613
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 6.896669864654541 | KNN Loss: 5.802331447601318 | BCE Loss: 1.0943384170532227
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 6.820957183837891 | KNN Loss: 5.726874828338623 | BCE Loss: 1.0940823554992676
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 6.794429779052734 | KNN Loss: 5.678097248077393 | BCE Loss: 1.1

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 6.182769298553467 | KNN Loss: 5.102409362792969 | BCE Loss: 1.0803598165512085
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 6.118555068969727 | KNN Loss: 5.076013088226318 | BCE Loss: 1.042541742324829
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 6.144984245300293 | KNN Loss: 5.0769243240356445 | BCE Loss: 1.0680601596832275
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 6.151422500610352 | KNN Loss: 5.071323394775391 | BCE Loss: 1.080099105834961
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 6.122405052185059 | KNN Loss: 5.060118675231934 | BCE Loss: 1.062286138534546
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 6.124342918395996 | KNN Loss: 5.078787326812744 | BCE Loss: 1.0455553531646729
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 6.1400980949401855 | KNN Loss: 5.086660861968994 | BCE Loss: 1.0534371137619019
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 6.175093173980713 | KNN Loss: 5.101563930511475 | BCE Loss

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 6.176115989685059 | KNN Loss: 5.059687614440918 | BCE Loss: 1.1164283752441406
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 6.091367721557617 | KNN Loss: 5.044297218322754 | BCE Loss: 1.0470705032348633
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 6.087880611419678 | KNN Loss: 5.050999641418457 | BCE Loss: 1.0368809700012207
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 6.07302188873291 | KNN Loss: 5.0403151512146 | BCE Loss: 1.0327069759368896
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 6.10439395904541 | KNN Loss: 5.044567108154297 | BCE Loss: 1.0598266124725342
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 6.067069053649902 | KNN Loss: 5.060004711151123 | BCE Loss: 1.0070645809173584
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 6.131958484649658 | KNN Loss: 5.058707237243652 | BCE Loss: 1.0732513666152954
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 6.127476692199707 | KNN Loss: 5.06211519241333 | BCE Loss: 1.

Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 6.08839225769043 | KNN Loss: 5.038623809814453 | BCE Loss: 1.0497686862945557
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 6.06320333480835 | KNN Loss: 5.032927989959717 | BCE Loss: 1.0302753448486328
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 6.117987632751465 | KNN Loss: 5.043788909912109 | BCE Loss: 1.0741989612579346
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 6.118149280548096 | KNN Loss: 5.034641265869141 | BCE Loss: 1.0835081338882446
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 6.092503547668457 | KNN Loss: 5.027092933654785 | BCE Loss: 1.0654107332229614
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 6.100611686706543 | KNN Loss: 5.050936222076416 | BCE Loss: 1.0496752262115479
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 6.132219314575195 | KNN Loss: 5.0503973960876465 | BCE Loss: 1.0818217992782593
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 6.072502136230469 | KNN Loss: 5.048242568969727 | BCE Loss:

...

Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 6.048320770263672 | KNN Loss: 5.015078544616699 | BCE Loss: 1.033242106437683
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 6.057239532470703 | KNN Loss: 5.008315563201904 | BCE Loss: 1.0489239692687988
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 6.038585662841797 | KNN Loss: 5.007270336151123 | BCE Loss: 1.0313154458999634
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 6.0623016357421875 | KNN Loss: 5.033756256103516 | BCE Loss: 1.028545618057251
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 6.041375160217285 | KNN Loss: 5.020190715789795 | BCE Loss: 1.0211846828460693
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 6.0682454109191895 | KNN Loss: 5.044124126434326 | BCE Loss: 1.0241211652755737
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 6.0270466804504395 | KNN Loss: 5.012785911560059 | BCE Loss: 1.0142606496810913
Epoch 450 / 500 | iteration 10 / 30 | Total Loss: 6.047994136810303 | KNN Loss: 5.017503261566162 |

Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 6.041540145874023 | KNN Loss: 5.014594078063965 | BCE Loss: 1.0269463062286377
Epoch   460: reducing learning rate of group 0 to 3.8655e-08.
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 6.061495304107666 | KNN Loss: 5.018918514251709 | BCE Loss: 1.042576789855957
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 6.020200252532959 | KNN Loss: 5.017342567443848 | BCE Loss: 1.0028576850891113
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 6.063215255737305 | KNN Loss: 5.003366470336914 | BCE Loss: 1.0598487854003906
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 6.055025100708008 | KNN Loss: 5.043058395385742 | BCE Loss: 1.0119664669036865
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 6.0618367195129395 | KNN Loss: 5.034313201904297 | BCE Loss: 1.0275235176086426
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 6.017180919647217 | KNN Loss: 4.99492883682251 | BCE Loss: 1.022252082824707
Epoch 461 / 500 | iteration 0 / 30 | To

Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 6.034457206726074 | KNN Loss: 4.995636940002441 | BCE Loss: 1.038820505142212
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 6.063051700592041 | KNN Loss: 5.032251358032227 | BCE Loss: 1.0308003425598145
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 6.077188491821289 | KNN Loss: 5.054178237915039 | BCE Loss: 1.023010492324829
Epoch   471: reducing learning rate of group 0 to 2.7058e-08.
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 6.084349632263184 | KNN Loss: 5.0446295738220215 | BCE Loss: 1.039720058441162
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 6.040957927703857 | KNN Loss: 5.012941360473633 | BCE Loss: 1.0280165672302246
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 6.046894550323486 | KNN Loss: 5.032792091369629 | BCE Loss: 1.0141024589538574
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 6.071711540222168 | KNN Loss: 5.06589412689209 | BCE Loss: 1.0058176517486572
Epoch 471 / 500 | iteration 20 / 30 | To

Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 6.049658298492432 | KNN Loss: 5.0222487449646 | BCE Loss: 1.027409553527832
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 6.035114288330078 | KNN Loss: 5.024486541748047 | BCE Loss: 1.0106276273727417
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 6.125916481018066 | KNN Loss: 5.033674716949463 | BCE Loss: 1.0922417640686035
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 6.008894443511963 | KNN Loss: 4.996998310089111 | BCE Loss: 1.0118961334228516
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 6.024177551269531 | KNN Loss: 5.020304203033447 | BCE Loss: 1.0038731098175049
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 6.044114112854004 | KNN Loss: 5.003721237182617 | BCE Loss: 1.0403928756713867
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 6.095734596252441 | KNN Loss: 5.062690734863281 | BCE Loss: 1.0330440998077393
Epoch 482 / 500 | iteration 10 / 30 | Total Loss: 6.028258323669434 | KNN Loss: 5.004518032073975 | BCE

Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 6.071241855621338 | KNN Loss: 5.025339126586914 | BCE Loss: 1.0459027290344238
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 6.095396041870117 | KNN Loss: 5.031296253204346 | BCE Loss: 1.0640995502471924
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 6.084624767303467 | KNN Loss: 5.0562663078308105 | BCE Loss: 1.0283584594726562
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 6.022395133972168 | KNN Loss: 5.016458511352539 | BCE Loss: 1.005936622619629
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 6.068243026733398 | KNN Loss: 5.025578498840332 | BCE Loss: 1.0426645278930664
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 6.061152458190918 | KNN Loss: 5.007332801818848 | BCE Loss: 1.0538197755813599
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 6.076411247253418 | KNN Loss: 5.056914329528809 | BCE Loss: 1.0194966793060303
Epoch 493 / 500 | iteration 0 / 30 | Total Loss: 6.053962707519531 | KNN Loss: 5.027441024780273 | 

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 3.0169,  3.7089,  2.5876,  3.0771,  3.3849,  0.7724,  2.6627,  2.2804,
          2.2103,  2.0220,  2.1847,  2.2892,  0.7973,  1.7530,  1.2190,  1.4315,
          2.7459,  2.7280,  2.2349,  1.8349,  1.7019,  2.5745,  2.2448,  2.6221,
          2.5252,  1.7932,  2.1448,  1.4807,  1.5369,  0.4111, -0.1998,  1.0286,
          0.2046,  0.9527,  1.4791,  1.4321,  1.0842,  2.8008,  0.6504,  1.2694,
          0.9280, -0.6987, -0.2524,  2.3327,  1.7224,  0.7449, -0.1590,  0.1547,
          1.4286,  2.5432,  1.3626,  0.1273,  1.3694,  0.5756, -0.5356,  1.0705,
          1.4649,  1.3859,  1.3082,  1.8057,  0.5729,  0.8208,  0.1695,  1.7984,
          1.3383,  1.7248, -1.7942,  0.3608,  2.3759,  2.2260,  2.4704,  0.4697,
          1.3686,  2.4387,  1.9007,  1.1780,  0.3153,  0.7410,  0.2198,  1.5819,
          0.0308,  0.4000,  1.8466, -0.3640,  0.2809, -1.0456, -2.3745, -0.1721,
          0.4682, -1.8568,  0.5005, -0.0813, -0.4984, -1.0866,  0.5996,  1.2267,
         -0.7037, -0.6617,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [12]:
dataset_ = [d[0].cpu() for d in dataset]

In [13]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 86.80it/s]


In [14]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [15]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [16]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [18]:
tensor_dataset = torch.stack(dataset_)

In [19]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [20]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [21]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [22]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [23]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [24]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [25]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [26]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [27]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [28]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [29]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [30]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [31]:
losses = []
accs = []
sparsity = []

In [32]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 029 | Total loss: 9.645 | Reg loss: 0.009 | Tree loss: 9.645 | Accuracy: 0.000000 | 0.291 sec/iter
Epoch: 00 | Batch: 001 / 029 | Total loss: 9.629 | Reg loss: 0.009 | Tree loss: 9.629 | Accuracy: 0.000000 | 0.262 sec/iter
Epoch: 00 | Batch: 002 / 029 | Total loss: 9.620 | Reg loss: 0.008 | Tree loss: 9.620 | Accuracy: 0.000000 | 0.248 sec/iter
Epoch: 00 | Batch: 003 / 029 | Total loss: 9.608 | Reg loss: 0.008 | Tree loss: 9.608 | Accuracy: 0.000000 | 0.242 sec/iter
Epoch: 00 | Batch: 004 / 029 | Total loss: 9.592 | Reg loss: 0.008 | Tree loss: 9.592 | Accuracy: 0.000000 | 0.24 sec/iter
Epoch: 00 | Batch: 005 / 029 | Total loss: 9.586 | Reg loss: 0.008 | Tree loss: 9.586 | Accuracy: 0.000000 | 0.239 sec/iter
Epoch: 00 | Batch: 006 / 029 | Total loss: 9.574 | Reg loss: 0.007 | Tree loss: 9.574 | Accuracy: 0.000000 | 0.238 sec/iter
Epoch: 00 | Batch:

Epoch: 02 | Batch: 004 / 029 | Total loss: 9.271 | Reg loss: 0.008 | Tree loss: 9.271 | Accuracy: 0.177734 | 0.238 sec/iter
Epoch: 02 | Batch: 005 / 029 | Total loss: 9.249 | Reg loss: 0.008 | Tree loss: 9.249 | Accuracy: 0.187500 | 0.238 sec/iter
Epoch: 02 | Batch: 006 / 029 | Total loss: 9.252 | Reg loss: 0.008 | Tree loss: 9.252 | Accuracy: 0.195312 | 0.238 sec/iter
Epoch: 02 | Batch: 007 / 029 | Total loss: 9.249 | Reg loss: 0.009 | Tree loss: 9.249 | Accuracy: 0.191406 | 0.237 sec/iter
Epoch: 02 | Batch: 008 / 029 | Total loss: 9.213 | Reg loss: 0.009 | Tree loss: 9.213 | Accuracy: 0.220703 | 0.237 sec/iter
Epoch: 02 | Batch: 009 / 029 | Total loss: 9.202 | Reg loss: 0.009 | Tree loss: 9.202 | Accuracy: 0.205078 | 0.237 sec/iter
Epoch: 02 | Batch: 010 / 029 | Total loss: 9.199 | Reg loss: 0.010 | Tree loss: 9.199 | Accuracy: 0.199219 | 0.237 sec/iter
Epoch: 02 | Batch: 011 / 029 | Total loss: 9.189 | Reg loss: 0.010 | Tree loss: 9.189 | Accuracy: 0.181641 | 0.237 sec/iter
Epoch: 0

Epoch: 04 | Batch: 009 / 029 | Total loss: 8.869 | Reg loss: 0.014 | Tree loss: 8.869 | Accuracy: 0.193359 | 0.237 sec/iter
Epoch: 04 | Batch: 010 / 029 | Total loss: 8.850 | Reg loss: 0.015 | Tree loss: 8.850 | Accuracy: 0.191406 | 0.237 sec/iter
Epoch: 04 | Batch: 011 / 029 | Total loss: 8.814 | Reg loss: 0.015 | Tree loss: 8.814 | Accuracy: 0.214844 | 0.237 sec/iter
Epoch: 04 | Batch: 012 / 029 | Total loss: 8.806 | Reg loss: 0.015 | Tree loss: 8.806 | Accuracy: 0.218750 | 0.237 sec/iter
Epoch: 04 | Batch: 013 / 029 | Total loss: 8.788 | Reg loss: 0.016 | Tree loss: 8.788 | Accuracy: 0.191406 | 0.237 sec/iter
Epoch: 04 | Batch: 014 / 029 | Total loss: 8.774 | Reg loss: 0.016 | Tree loss: 8.774 | Accuracy: 0.236328 | 0.237 sec/iter
Epoch: 04 | Batch: 015 / 029 | Total loss: 8.762 | Reg loss: 0.017 | Tree loss: 8.762 | Accuracy: 0.224609 | 0.236 sec/iter
Epoch: 04 | Batch: 016 / 029 | Total loss: 8.756 | Reg loss: 0.017 | Tree loss: 8.756 | Accuracy: 0.246094 | 0.236 sec/iter
Epoch: 0

Epoch: 06 | Batch: 014 / 029 | Total loss: 8.319 | Reg loss: 0.020 | Tree loss: 8.319 | Accuracy: 0.314453 | 0.237 sec/iter
Epoch: 06 | Batch: 015 / 029 | Total loss: 8.297 | Reg loss: 0.020 | Tree loss: 8.297 | Accuracy: 0.298828 | 0.237 sec/iter
Epoch: 06 | Batch: 016 / 029 | Total loss: 8.303 | Reg loss: 0.021 | Tree loss: 8.303 | Accuracy: 0.283203 | 0.237 sec/iter
Epoch: 06 | Batch: 017 / 029 | Total loss: 8.281 | Reg loss: 0.021 | Tree loss: 8.281 | Accuracy: 0.263672 | 0.237 sec/iter
Epoch: 06 | Batch: 018 / 029 | Total loss: 8.255 | Reg loss: 0.022 | Tree loss: 8.255 | Accuracy: 0.304688 | 0.237 sec/iter
Epoch: 06 | Batch: 019 / 029 | Total loss: 8.248 | Reg loss: 0.022 | Tree loss: 8.248 | Accuracy: 0.302734 | 0.237 sec/iter
Epoch: 06 | Batch: 020 / 029 | Total loss: 8.234 | Reg loss: 0.022 | Tree loss: 8.234 | Accuracy: 0.304688 | 0.237 sec/iter
Epoch: 06 | Batch: 021 / 029 | Total loss: 8.214 | Reg loss: 0.023 | Tree loss: 8.214 | Accuracy: 0.302734 | 0.237 sec/iter
Epoch: 0

Epoch: 08 | Batch: 019 / 029 | Total loss: 7.618 | Reg loss: 0.025 | Tree loss: 7.618 | Accuracy: 0.300781 | 0.238 sec/iter
Epoch: 08 | Batch: 020 / 029 | Total loss: 7.597 | Reg loss: 0.026 | Tree loss: 7.597 | Accuracy: 0.283203 | 0.238 sec/iter
Epoch: 08 | Batch: 021 / 029 | Total loss: 7.554 | Reg loss: 0.026 | Tree loss: 7.554 | Accuracy: 0.328125 | 0.238 sec/iter
Epoch: 08 | Batch: 022 / 029 | Total loss: 7.569 | Reg loss: 0.026 | Tree loss: 7.569 | Accuracy: 0.310547 | 0.238 sec/iter
Epoch: 08 | Batch: 023 / 029 | Total loss: 7.570 | Reg loss: 0.027 | Tree loss: 7.570 | Accuracy: 0.300781 | 0.238 sec/iter
Epoch: 08 | Batch: 024 / 029 | Total loss: 7.517 | Reg loss: 0.027 | Tree loss: 7.517 | Accuracy: 0.320312 | 0.238 sec/iter
Epoch: 08 | Batch: 025 / 029 | Total loss: 7.483 | Reg loss: 0.027 | Tree loss: 7.483 | Accuracy: 0.328125 | 0.238 sec/iter
Epoch: 08 | Batch: 026 / 029 | Total loss: 7.507 | Reg loss: 0.028 | Tree loss: 7.507 | Accuracy: 0.242188 | 0.238 sec/iter
Epoch: 0

...

Epoch: 91 | Batch: 000 / 029 | Total loss: 1.791 | Reg loss: 0.026 | Tree loss: 1.791 | Accuracy: 0.324219 | 0.24 sec/iter
Epoch: 91 | Batch: 001 / 029 | Total loss: 1.819 | Reg loss: 0.026 | Tree loss: 1.819 | Accuracy: 0.310547 | 0.24 sec/iter
Epoch: 91 | Batch: 002 / 029 | Total loss: 1.809 | Reg loss: 0.026 | Tree loss: 1.809 | Accuracy: 0.324219 | 0.24 sec/iter
Epoch: 91 | Batch: 003 / 029 | Total loss: 1.769 | Reg loss: 0.026 | Tree loss: 1.769 | Accuracy: 0.337891 | 0.24 sec/iter
Epoch: 91 | Batch: 004 / 029 | Total loss: 1.759 | Reg loss: 0.026 | Tree loss: 1.759 | Accuracy: 0.316406 | 0.24 sec/iter
Epoch: 91 | Batch: 005 / 029 | Total loss: 1.814 | Reg loss: 0.026 | Tree loss: 1.814 | Accuracy: 0.298828 | 0.24 sec/iter
Epoch: 91 | Batch: 006 / 029 | Total loss: 1.800 | Reg loss: 0.026 | Tree loss: 1.800 | Accuracy: 0.298828 | 0.24 sec/iter
Epoch: 91 | Batch: 007 / 029 | Total loss: 1.741 | Reg loss: 0.026 | Tree loss: 1.741 | Accuracy: 0.320312 | 0.24 sec/iter
Epoch: 91 | Batc

Epoch: 93 | Batch: 005 / 029 | Total loss: 1.769 | Reg loss: 0.026 | Tree loss: 1.769 | Accuracy: 0.296875 | 0.239 sec/iter
Epoch: 93 | Batch: 006 / 029 | Total loss: 1.795 | Reg loss: 0.026 | Tree loss: 1.795 | Accuracy: 0.289062 | 0.239 sec/iter
Epoch: 93 | Batch: 007 / 029 | Total loss: 1.819 | Reg loss: 0.026 | Tree loss: 1.819 | Accuracy: 0.292969 | 0.239 sec/iter
Epoch: 93 | Batch: 008 / 029 | Total loss: 1.733 | Reg loss: 0.026 | Tree loss: 1.733 | Accuracy: 0.335938 | 0.239 sec/iter
Epoch: 93 | Batch: 009 / 029 | Total loss: 1.749 | Reg loss: 0.026 | Tree loss: 1.749 | Accuracy: 0.324219 | 0.239 sec/iter
Epoch: 93 | Batch: 010 / 029 | Total loss: 1.792 | Reg loss: 0.026 | Tree loss: 1.792 | Accuracy: 0.312500 | 0.239 sec/iter
Epoch: 93 | Batch: 011 / 029 | Total loss: 1.755 | Reg loss: 0.026 | Tree loss: 1.755 | Accuracy: 0.312500 | 0.239 sec/iter
Epoch: 93 | Batch: 012 / 029 | Total loss: 1.740 | Reg loss: 0.026 | Tree loss: 1.740 | Accuracy: 0.283203 | 0.239 sec/iter
Epoch: 9

Epoch: 95 | Batch: 010 / 029 | Total loss: 1.693 | Reg loss: 0.026 | Tree loss: 1.693 | Accuracy: 0.318359 | 0.239 sec/iter
Epoch: 95 | Batch: 011 / 029 | Total loss: 1.722 | Reg loss: 0.026 | Tree loss: 1.722 | Accuracy: 0.308594 | 0.239 sec/iter
Epoch: 95 | Batch: 012 / 029 | Total loss: 1.758 | Reg loss: 0.026 | Tree loss: 1.758 | Accuracy: 0.353516 | 0.239 sec/iter
Epoch: 95 | Batch: 013 / 029 | Total loss: 1.721 | Reg loss: 0.026 | Tree loss: 1.721 | Accuracy: 0.287109 | 0.239 sec/iter
Epoch: 95 | Batch: 014 / 029 | Total loss: 1.742 | Reg loss: 0.026 | Tree loss: 1.742 | Accuracy: 0.318359 | 0.239 sec/iter
Epoch: 95 | Batch: 015 / 029 | Total loss: 1.730 | Reg loss: 0.026 | Tree loss: 1.730 | Accuracy: 0.316406 | 0.239 sec/iter
Epoch: 95 | Batch: 016 / 029 | Total loss: 1.737 | Reg loss: 0.026 | Tree loss: 1.737 | Accuracy: 0.308594 | 0.239 sec/iter
Epoch: 95 | Batch: 017 / 029 | Total loss: 1.705 | Reg loss: 0.026 | Tree loss: 1.705 | Accuracy: 0.294922 | 0.239 sec/iter
Epoch: 9

Epoch: 97 | Batch: 015 / 029 | Total loss: 1.769 | Reg loss: 0.026 | Tree loss: 1.769 | Accuracy: 0.285156 | 0.239 sec/iter
Epoch: 97 | Batch: 016 / 029 | Total loss: 1.693 | Reg loss: 0.026 | Tree loss: 1.693 | Accuracy: 0.328125 | 0.239 sec/iter
Epoch: 97 | Batch: 017 / 029 | Total loss: 1.747 | Reg loss: 0.026 | Tree loss: 1.747 | Accuracy: 0.302734 | 0.239 sec/iter
Epoch: 97 | Batch: 018 / 029 | Total loss: 1.723 | Reg loss: 0.026 | Tree loss: 1.723 | Accuracy: 0.316406 | 0.239 sec/iter
Epoch: 97 | Batch: 019 / 029 | Total loss: 1.697 | Reg loss: 0.026 | Tree loss: 1.697 | Accuracy: 0.347656 | 0.239 sec/iter
Epoch: 97 | Batch: 020 / 029 | Total loss: 1.697 | Reg loss: 0.026 | Tree loss: 1.697 | Accuracy: 0.289062 | 0.239 sec/iter
Epoch: 97 | Batch: 021 / 029 | Total loss: 1.627 | Reg loss: 0.026 | Tree loss: 1.627 | Accuracy: 0.359375 | 0.239 sec/iter
Epoch: 97 | Batch: 022 / 029 | Total loss: 1.644 | Reg loss: 0.026 | Tree loss: 1.644 | Accuracy: 0.337891 | 0.239 sec/iter
Epoch: 9

Epoch: 99 | Batch: 020 / 029 | Total loss: 1.726 | Reg loss: 0.026 | Tree loss: 1.726 | Accuracy: 0.279297 | 0.239 sec/iter
Epoch: 99 | Batch: 021 / 029 | Total loss: 1.719 | Reg loss: 0.026 | Tree loss: 1.719 | Accuracy: 0.330078 | 0.239 sec/iter
Epoch: 99 | Batch: 022 / 029 | Total loss: 1.674 | Reg loss: 0.026 | Tree loss: 1.674 | Accuracy: 0.339844 | 0.239 sec/iter
Epoch: 99 | Batch: 023 / 029 | Total loss: 1.735 | Reg loss: 0.026 | Tree loss: 1.735 | Accuracy: 0.287109 | 0.239 sec/iter
Epoch: 99 | Batch: 024 / 029 | Total loss: 1.682 | Reg loss: 0.026 | Tree loss: 1.682 | Accuracy: 0.330078 | 0.239 sec/iter
Epoch: 99 | Batch: 025 / 029 | Total loss: 1.694 | Reg loss: 0.026 | Tree loss: 1.694 | Accuracy: 0.322266 | 0.239 sec/iter
Epoch: 99 | Batch: 026 / 029 | Total loss: 1.732 | Reg loss: 0.026 | Tree loss: 1.732 | Accuracy: 0.281250 | 0.239 sec/iter
Epoch: 99 | Batch: 027 / 029 | Total loss: 1.698 | Reg loss: 0.026 | Tree loss: 1.698 | Accuracy: 0.314453 | 0.239 sec/iter
Epoch: 9

In [33]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [35]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 7.048192771084337


# Extract Rules

# Accumulate samples in the leaves

In [36]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 83


In [37]:
method = 'greedy'

In [38]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [39]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 32 ==============
============== Pattern 33 ==============
============== Pattern 34 ==============
============== Pattern 35 ==============
============== Pattern 36 ==============
============== Pattern 37 ==============
============== Pattern 38 ==============
============== Pattern 39 ==============
============== Pattern 40 ==============
============== Pattern 41 ==============
============== Pattern 42 ==============
============== Pattern 43 ==============
============== Pattern 44 ==============
============== Pattern 45 ==============
============== Pattern 46 ==============
============== Pattern 47 ==============
============== Pattern 48 ==============
============== Pattern 49 ==============
============== Pattern 50 ==============
============== Pattern 51 ==============
============== Pattern 52 ==============
============== Pattern 53 ==============
============== Pattern 54 ==============
============== Pattern 55 ==============
============== P